In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torchvision
from sklearn.metrics import confusion_matrix  
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import VisionDataset
from torch.utils.data import Subset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
import os
import os.path
import sys
from torch.backends import cudnn
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

from datasets import Cityscapes
from clients import *
from servers import *
from utils import *
from models import *


In [3]:
#DEVICE = 'cuda' # 'cuda' or 'cpu'
DEVICE = 'cuda'
NUM_CLASSES = 19

BATCH_SIZE = 16     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 5*1e-3           # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  

NUM_EPOCHS = 25     
STEP_SIZE = 20       #20 How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

In [4]:
#transformations
size = (256, 512) #512,1024
scales=(0.25, 2.)
cropsize=(512, 1024)
eval_crop=(1024, 1024)
eval_scales=(0.5, 0.75, 1.0, 1.25, 1.5, 1.75)
test_transformations = [RandomResizedCrop(eval_scales, eval_crop)]
train_transformations = [RandomResizedCrop(scales,cropsize),RandomHorizontalFlip(),ColorJitter(brightness=0.4,contrast=0.4,saturation=0.4)]
test_transformations = Compose(test_transformations)
train_transformations = Compose(train_transformations)

In [5]:

clients = []
datasets = []
styles = []
with open("/content/drive/MyDrive/data/data/Cityscapes/train_A_I.txt") as lines:
  curr_id = 0
  paths = []
  for line in lines:
    id_client,path = line.split(" ")
    if curr_id == int(id_client):
      paths.append(path)
    else:
      print(f"finished Client ID:[{curr_id}]: len: {len(paths)}")
      dataset = DatasetClient("/content/drive/MyDrive/data/data/Cityscapes",paths,train_transformations)
      datasets.append(dataset)
      client = Client(curr_id,dataset)
      clients.append(client)

      paths = []
      paths.append(path)
      curr_id = int(id_client)  

  dataset = DatasetClient("/content/drive/MyDrive/data/data/Cityscapes",paths,train_transformations)
  datasets.append(dataset)
  client = Client(curr_id,dataset)
  clients.append(client)
  client.compute_avg_style()
  styles.append(deepcopy(client.style_detector))
  
#print(len(clients), f"last: {curr_id}")
i,_ = datasets[0].__getitem__(0)
print(i.shape)

finished Client ID:[0]: len: 20
finished Client ID:[1]: len: 20
finished Client ID:[2]: len: 20
finished Client ID:[3]: len: 20
finished Client ID:[4]: len: 20
finished Client ID:[5]: len: 20
finished Client ID:[6]: len: 20
finished Client ID:[7]: len: 20
finished Client ID:[8]: len: 20
finished Client ID:[9]: len: 20
finished Client ID:[10]: len: 20
finished Client ID:[11]: len: 20
finished Client ID:[12]: len: 20
finished Client ID:[13]: len: 20
finished Client ID:[14]: len: 20
finished Client ID:[15]: len: 20
finished Client ID:[16]: len: 20
finished Client ID:[17]: len: 20
finished Client ID:[18]: len: 20
finished Client ID:[19]: len: 20
finished Client ID:[20]: len: 20
finished Client ID:[21]: len: 20
finished Client ID:[22]: len: 20
finished Client ID:[23]: len: 20
finished Client ID:[24]: len: 20
finished Client ID:[25]: len: 20
finished Client ID:[26]: len: 20
finished Client ID:[27]: len: 20
finished Client ID:[28]: len: 20
finished Client ID:[29]: len: 20
finished Client ID:[

100%|██████████| 8/8 [00:03<00:00,  2.29it/s]


torch.Size([3, 512, 1024])


In [6]:
dataset_GTA = GTA5(root="/content/drive/MyDrive/data/data/GTA5",transform=train_transformations,fda=True,styles=styles)
train_dataloader = DataLoader(dataset_GTA, batch_size=BATCH_SIZE, shuffle=True, num_workers=2,drop_last=True)

In [7]:
test_dataset_B = Cityscapes('/content/drive/MyDrive/data/data/Cityscapes',transform=None,split='val')
test_dataset_A = Cityscapes('/content/drive/MyDrive/data/data/Cityscapes',transform=None,split='valA')

In [8]:
test_dataloader_B = DataLoader(test_dataset_B, batch_size=1, shuffle=False, num_workers=2,drop_last=True)
test_dataloader_A = DataLoader(test_dataset_A, batch_size=1, shuffle=False, num_workers=2,drop_last=True)

In [ ]:
server = Server_FDA(clients,train_dataloader=train_dataloader,test_dataloaderA=test_dataloader_A,test_dataloaderB=test_dataloader_B,styles=styles)

server.train()

